In [42]:
!pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/f6/26/f890297cbd5e7f51bf912cd8149fc21f6930d3559a24f7be3d8cd77d8933/PyMuPDF-1.24.10-cp38-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.24.10 from https://files.pythonhosted.org/packages/16/7a/f634c76d8331cb8dedcfaced17424cc469ee20b7f53cf29c9ef17a01b461/PyMuPDFb-1.24.10-py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 36.4 MB/s eta 0:00:0000:0100:01


In [29]:
#############
# LIBRARIES #
#############

import pandas as pd

import docx
from docx.opc.constants import RELATIONSHIP_TYPE as RT

import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import time

In [30]:
# readi_resources_doc_path = '/Users/rogerlefort/Library/CloudStorage/OneDrive-Personal/Documents/New_ReaDI_Page_Layout.docx'
test_page = 'test_page.docx'

def extract_hyperlinks(docx_file):
    doc = docx.Document(docx_file)
    rels = doc.part.rels
    hyperlinks = []

    for rel in rels:
        if rels[rel].reltype == RT.HYPERLINK:
            hyperlinks.append(rels[rel]._target)

    return hyperlinks

links = extract_hyperlinks(test_page)
for link in links:
    print(link)

https://www.grants.gov/learn-grants/grants-101/the-grant-lifecycle


In [31]:
# Function to check if a URL is valid
def is_valid_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        return response.status_code == 200
    except requests.RequestException:
        return False

# Check if each link is valid
valid_links = [link for link in links if is_valid_url(link)]

# Print the valid links
for link in valid_links:
    print(link)

https://www.grants.gov/learn-grants/grants-101/the-grant-lifecycle


In [38]:
def analyze_links_with_screenshots(links, screenshot_dir):
    results = []
    
    # Set up headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    for i, link in enumerate(links):
        try:
            # Check if the link returns content
            response = requests.get(link, timeout=10)
            status_code = response.status_code
            content_returned = len(response.content) > 0

            # Capture screenshot
            driver.get(link)
            time.sleep(5)  # Wait for page to load
            screenshot_path = os.path.join(screenshot_dir, f"screenshot_{i}.png")
            driver.save_screenshot(screenshot_path)

            results.append({
                "link": link,
                "status_code": status_code,
                "content_returned": content_returned,
                "screenshot_path": screenshot_path
            })
        except Exception as e:
            results.append({
                "link": link,
                "error": str(e)
            })

    driver.quit()
    return results

# Example usage
links = extract_hyperlinks("test_page.docx")
screenshot_dir = "output"
os.makedirs(screenshot_dir, exist_ok=True)
analysis_results = analyze_links_with_screenshots(links, screenshot_dir)

for result in analysis_results:
    print(f"Link: {result['link']}")
    if 'error' in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Status Code: {result['status_code']}")
        print(f"Content Returned: {result['content_returned']}")
        print(f"Screenshot: {result['screenshot_path']}")
    # Create a dataframe with the results
    df = pd.DataFrame(analysis_results)
    # Save the dataframe to a csv file
    df.to_csv("analysis_results.csv", index=False)
    print()

Link: https://www.grants.gov/learn-grants/grants-101/the-grant-lifecycle
Status Code: 200
Content Returned: True
Screenshot: output/screenshot_0.png



In [46]:
import fitz  # PyMuPDF
import re

def extract_hyperlinks_directly(pdf_file_path):
    hyperlinks = []
    pdf_document = fitz.open(pdf_file_path)
    
    # Regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://[^\s]+')
    
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text = page.get_text("text")  # Extract the text from the page
        links = re.findall(url_pattern, text)  # Find all URLs in the text
        hyperlinks.extend(links)
    
    return hyperlinks

# Path to your PDF file
pdf_file_path = "test_page.pdf"

# Extract hyperlinks
extracted_hyperlinks = extract_hyperlinks_directly(pdf_file_path)

# Print the extracted hyperlinks
for hyperlink in extracted_hyperlinks:
    print(hyperlink)


In [47]:
extracted_hyperlinks

[]